In [67]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn
seaborn.set()
import scipy
import os
import glob
import time

In [76]:
import time
start_time = time.time()

time.sleep(1)
print("{:.3f} ms to run".format(1000* (time.time() - start_time)))

1006.774 ms to run


In [3]:
import Master as M

Client = M.Make_Client('10.50.0.10')

Result = M.Read(Client, 22201)

print(Result)

0.5666666626930237
Wall time: 72.8 ms


In [73]:
###############################################################################
###############################################################################
# Author: Austin Czyzewski
# Date: 06/25/2020; Date Tested: 06/25/2020
#
# Purpose: Take amplifier status data and push into PLC. Reduce need for 
#           conversions, have more readily accessible data, etc.
#
# Method:
#       1- Connect to PLC, Connect to Amplifier
#       2- Read XML file from Amplifier, import as list
#       3- String format to get rid of HTML + XML overhead
#       4- Store Names and Values
#       5- Write Values to the PLC with pre-defined modbus addresses
#       6- Go back to step 2
#
###############################################################################
###############################################################################


#############################
## imports
#############################

import time #import our time module to allow us to wait
import os #allow us to interface with command line prompts like curl
import numpy as np #math functions
import Master as M #importing the PLC communications library

#############################
## Connect to PLC, define Modbus addresses
#############################

#Client = M.Make_Client('10.1.2.100') #Connecting to PLC

Conversion_Rate = 50 #Convert amp forward power from percent to Watts

filler = 1001 #Temporary puppy
Tag_List = np.arange(50001,50100,2) #List of tags in order of writing
                                    #Only one really needed
           
#############################
## Begin reading loop
#############################
while True:
    start_time = time.time() #taking the time to evaluate how long the process 
                                #takes to run
                                
    #############################
    ## Grab the data from the amplifier; make more usable
    #############################

    #Amp_Response = os.popen("curl -u admin:admin http://169.254.1.1/status.xml") #5KW off network
    
    #Amp_Response = os.popen("curl -u admin:admin http://10.50.0.21/status.xml") #West Tunnel
    
    Amp_Response = os.popen("curl -u admin:admin http://10.1.2.125/status.xml")
        #Above grabs data from xml file
        
    amp = list() #Create an empty list 
    
    for line in Amp_Response: #iterate over each value of the Amp Response
        
        amp.append(line.split('>')) #Split into two lines at the '>' in 
                                #<Name>Value<Name> format, creates 2 pieces

    Names = list() #Create empty lists of the names and values of each tag as 
    Values = list() #defined above
    
    for num, i in enumerate(amp): #For the index and each value in the list 
                                    #of responses
            
        amp[num][0] = amp[num][0].strip('<') #Get rid of the '<' in the Name 
                                                #value for aesthetic
            
        Names.append(amp[num][0]) #Add the now beautified name to the list 
                                    #of Names
            
        try: #Here we are going to try to do the following function but 
                #since some lines contain incorrect format, we have 
                #error handling in this form
                
            temp = i[1].split('<') #Split the second part of the string going 
                                    #from Value<Name to [Value, <Name]
                
            amp[num][1] = temp #Replace the Value<Name with list of 
                                #value and name
            if "!DEF" in amp[num][1]:
                Values.append("NaN")
            else:
                Values.append(amp[num][1][0]) #Only append the first value in the 
                                            #list [Value, <Name] to get Value
                
        except: #If our format is not in Value<Name then we add 
            Values.append('NaN') #this arbitrary string as filler
            
    Values[-1] = str(round(float(Values[8]) * Conversion_Rate,3)) #Adding the converted
                                                        # FWD Power value
    Names[-1] = "MeasuredFWDWatts"
    
#     for it in range(len(Names)):
#         print(Names[it], Values[it])
#         if "!DEF" in Values[it]:
#             Values[it] = 'Nan'
            
        
    os.system('cls') #Clears the print screen to save on memory 
                        #(Not needed if only writing to PLCs)

    #############################
    ## Write to the PLC
    #############################
    
    Values = np.array([float(Value) for Value in Values[1:]])
    
    #M.Write_Multiple(Client, Tag_List[0], Values) #Writing all of the values to the PLC
    
    for Name, Value in zip(Names[1:], Values): #Here is filler for when we want 
                                            #to write these values to the PLC
            
    #    M.Write(Client, Tag_Num, float(Value)) #Writes to the Tag in Tag_List
        print(Name, Value)
        continue #Avoid parsing error with empty function for testing
        
    print("{:.3f} ms to run".format(1000* (time.time() - start_time))) 
    #Taking the current time and comparing to the start time. 
        #Printing the difference in ms
    
    #time.sleep(1) #sleep for designated amount of time in seconds
    


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current Not existant homie
                                 Dload  Upload   Total   Spent    Left  Speed Not existant homie
 Not existant homie
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0 Not existant homie
100  1657    0  1657    0     0   1657      0 --:--:-- --:--:-- --:--:--  3657 Not existant homie
response 
StsCarrier 1
StsOverdrive 1
StsVSWR 1
StsVolatge 1
StsRfIn 1
StsTemp 1
StsAGC 1
MeasuredFWD 7.5
MeasuredRFL 0.0
MeasuredVolts 23
MeasuredRfIn 193
measuredtemp 0
AGCSetPt 84
Attenuation 0
VSWRTripPt 8
StsCarrierPA1 1
StsOverdrivePA1 1
StsVSWRPA1 1
StsVolatgePA1 1
StsRfInPA1 1
StsTempPA1 1
StsCarrierPA2 1
StsOverdrivePA2 1
StsVSWRPA2 1
StsVolatgePA2 1
StsRfInPA2 1
StsTempPA2 1
MeasuredFWDPA1 5
MeasuredFWDPA2 5
MeasuredRFLPA1 0
MeasuredRFLPA2 0
MeasuredVoltsPA1 50
MeasuredVoltsPA2 50
MeasuredRfInPA1 4
MeasuredRfInPA2 4
measuredtempPA1 26
measuredtempPA2 26
MeasuredFnl1PA1 

KeyboardInterrupt: 

In [79]:
import numpy as np

tag_range = np.arange(50001,50100,2)
print(tag_range[1])


50003


In [68]:
amp = list()
with open("amp sample.txt", 'r') as file:
    for line in file:
        amp.append(line.split('>'))
        #amp.append(line.strip('\n').split('>'))


response 

StsCarrier 0
StsOverdrive 1
StsVSWR 1
StsVolatge 1
StsRfIn 1
StsTemp 1
StsAGC 1
MeasuredFWD 0.0
MeasuredRFL 0.0
MeasuredVolts 23
MeasuredRfIn 193
measuredtemp 0
AGCSetPt 84
Attenuation 39
VSWRTripPt 8
StsCarrierPA1 0
StsOverdrivePA1 1
StsVSWRPA1 1
StsVolatgePA1 1
StsRfInPA1 1
StsTempPA1 1
StsCarrierPA2 0
StsOverdrivePA2 1
StsVSWRPA2 1
StsVolatgePA2 1
StsRfInPA2 1
StsTempPA2 1
MeasuredFWDPA1 0
MeasuredFWDPA2 0
MeasuredRFLPA1 0
MeasuredRFLPA2 0
MeasuredVoltsPA1 50
MeasuredVoltsPA2 50
MeasuredRfInPA1 0
MeasuredRfInPA2 0
measuredtempPA1 24
measuredtempPA2 21
MeasuredFnl1PA1 0.4
MeasuredFnl1PA2 0.4
MeasuredFnl2PA1 0.3
MeasuredFnl2PA2 0.4
MeasuredFnl3PA1 0.1
MeasuredFnl3PA2 0.4
MeasuredDrv1PA1 0.0
MeasuredDrv1PA2 0.0
CurrentDrv1 !DEF
CurrentFnl1 !DEF
CurrentFnl2 !DEF
CurrentFnl3 !DEF
/response 
